In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate

class ViTFeatureExtractorModel(Model):
    def __init__(self, num_outputs):
        super(ViTFeatureExtractorModel, self).__init__()
        # Load the pre-trained ViT model from TensorFlow Hub
        self.vit_model = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_b16_fe/1", 
                                        trainable=False, input_shape=(224, 224, 3))
        
        # Define the model for the RGB input
        self.rgb_input = Input(shape=(224, 224, 3), name='rgb_input')
        # Define the model for the RGB depth input
        self.depth_input = Input(shape=(224, 224, 3), name='depth_input')
        
        # You can add additional layers here if needed
        # For example, Dense layers for further processing the concatenated features
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(num_outputs, activation='linear')  # Adjust based on your output needs

    def call(self, inputs):
        rgb_input, depth_input = inputs
        
        # Extract features using the ViT model
        rgb_features = self.vit_model(rgb_input)
        depth_features = self.vit_model(depth_input)
        
        # Concatenate features from both inputs
        concatenated_features = concatenate([rgb_features, depth_features])
        
        # Further processing and output
        x = self.dense1(concatenated_features)
        return self.dense2(x)

# Instantiate the model
num_outputs = 5  # For example, predicting 5 different properties
model = ViTFeatureExtractorModel(num_outputs=num_outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Or another appropriate loss function
              metrics=['accuracy'])


2024-03-21 10:52:41.175811: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 10:52:41.597167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 10:52:42.365480: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-21 10:52:43.787049: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

In [2]:

# To summarize the model, you need to build it first by calling it on some sample data
sample_rgb_input = tf.zeros([1, 224, 224, 3])
sample_depth_input = tf.zeros([1, 224, 224, 3])
model([sample_rgb_input, sample_depth_input])  # This builds the model
model.summary()

I0000 00:00:1711014766.674814  252381 service.cc:145] XLA service 0x1ab82990 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711014766.675009  252381 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-03-21 10:52:46.771967: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711014769.021719  252381 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model: "vi_t_feature_extractor_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,381 (771.02 KB)

 Trainable params: 197,381 (771.02 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import pandas as pd
import numpy as np
data = np.array(pd.read_pickle('df.pkl'))
depths = pd.read_pickle('depths.pkl')
images = pd.read_pickle('images.pkl')

In [4]:
X = np.array([[images[i], depths[i]] for i in range(len(images))])

In [5]:
del depths, images

In [6]:
a,b = X[0]

In [7]:
a.shape

(250, 250, 3)

In [8]:
history = model.fit(X, data, epochs=10, batch_size=32, validation_split=0.2)  # Example training loop

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [9]:
np.array([1,2])

array([1, 2])

In [10]:
g,h = np.array([1,2])